# Building a Supercharged Web Agent

We’ll be building a “Supercharged Research Agent” that takes a user prompt, searches the web, and returns a clean, structured summary for human use.

## Objective :
* Building an AI Agent for business research, competitive analysis, or real-time intelligence
* Building and Agnet which will use Web-search. Include web-serach into your LLM flow.
* We will be building a "Supercharge Research Agent" that takes a user prompt, searches the web and returns a clean, structured summary for human use


We will use OpenAI Hosted Tools

* OpenAi's Agents SDK provides Hosted Tools - pre-integrated, server-side fucntionalities accessble to any agent you build.

In [4]:
# Import Dependency Libs
from dotenv import load_dotenv
from agents import Agent, WebSearchTool, trace, Runner
from agents.model_settings import ModelSettings
from IPython.display import display, Markdown


load_dotenv(override=True)

# Agent Instructions: Clear and Direct
RESEARCH_PROMPT = (
    "You are a highly efficient research assistant. "
    "Perform a thorough web search and return a well-organized, factual summary "
    "in under 500 words. "
    "CRITICAL: For every key fact or data point, you must include a source URL "
    "in brackets immediately following the information (e.g., [https://example.com]). "
    "At the end of your report, provide a 'Sources' section listing all unique URLs used."
    "No personal opinions or commentary. This is for someone writing a report."
)

# Initialize Web Search Agent
web_search_agent = Agent(
    name="Web Research Agent",
    instructions=RESEARCH_PROMPT,
    tools=[WebSearchTool(search_context_size='low')],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required")

)

In [5]:
# Topic to Search
search="Best 3 performing index funds in the INdia market in current quarter"

# Run the Agent
with trace("Web Search Execution"):
    result = await Runner.run(web_search_agent, search)

# Display Results
display(Markdown(result.final_output))

As of February 9, 2026, the top-performing index funds in the Indian market for the current quarter are:

1. **Motilal Oswal Nifty Smallcap 250 Index Fund**: This fund has demonstrated a 5-year Compound Annual Growth Rate (CAGR) of 32.07% as of July 30, 2025. ([linkedin.com](https://www.linkedin.com/posts/smc-group_business-indexfund-activity-7356663443925463042-U41c?utm_source=openai))

2. **Sundaram Finance Nifty 100 Equal Weight Fund**: It has achieved a 5-year CAGR of 21.78% as of July 30, 2025. ([linkedin.com](https://www.linkedin.com/posts/smc-group_business-indexfund-activity-7356663443925463042-U41c?utm_source=openai))

3. **Motilal Oswal Nifty Bank Index Fund**: This fund reported a 5-year CAGR of 21.33% as of July 30, 2025. ([linkedin.com](https://www.linkedin.com/posts/smc-group_business-indexfund-activity-7356663443925463042-U41c?utm_source=openai))

Please note that these performance figures are based on data available up to July 30, 2025. For the most current performance metrics, it is advisable to consult the latest fund fact sheets or financial reports.

**Sources:**

- ([linkedin.com](https://www.linkedin.com/posts/smc-group_business-indexfund-activity-7356663443925463042-U41c?utm_source=openai)) 

### Pricing of OpenAI APIs

https://platform.openai.com/docs/pricing#web-search

### Check out the trace:

https://platform.openai.com/traces

# Planning with Precision : Structuring AI Output using Pydantic 

# Planning Web Searches with Structured AI Output

# Objective :
 * We will ask LLM based agent to plan a set of web searches to answers a financial query.
 * WHen building multi-toll agents , unstructured natural language can be ambiguous and hard to parse.
 * That's where Pydantic comes in :
  It allows you to define strict, validate, and typed outputs that your AI must follow.

* Instead of returning plain text, that agent returns structured objects with a clear (reason + query) for each search


In [13]:
from pydantic import BaseModel

class SearchTask(BaseModel):
    reason :str # Why this particular search matters 
    query : str # Tha actual web search string


# The full output schema from the planner
class SearchPlan(BaseModel):
    searches: list[SearchTask] # A list of search agents 


#  How many searches should the planner generate ?
NUMBER_OF_SEARCHES = 4

# Instruction : Well-formed prompt for planning 
PLANNER_PROMPT = (
    f"you are a financial research planner. Given a user query, identity {NUMBER_OF_SEARCHES} distinct search terms"
    "that would help answer it comprehensively. For each scenario, include a short reason explaininig its relevance"
)

# Agent to generate the structured search plan 
search_planner_agent = Agent(
                            name='SearchPlannerAgent',
                            instructions= PLANNER_PROMPT,
                            model='gpt-4o-mini',
                            output_type = SearchPlan # Agent is expected to output a Pydantic structure
                            )


# Example -financial query
user_query = "Best 3 performinig index funcds in the India market in quater"

with trace("SearchPlan Generation"):
    result = await Runner.run(search_planner_agent,user_query)
    print((result.final_output))

searches=[SearchTask(reason='This term will help identify the top-performing index funds in India, focusing specifically on their returns over the specified quarter.', query='top performing index funds India Q3 2023'), SearchTask(reason='This term allows for a deeper understanding of the market conditions and economic factors during the quarter that influenced index fund performance.', query='Indian market analysis Q3 2023'), SearchTask(reason='Exploring this term will provide insights into the criteria investors should use to evaluate index fund performance, such as expense ratios and tracking error.', query='how to evaluate index fund performance'), SearchTask(reason='This term will help identify any specific funds that have recently gained popularity or received accolades for their performance in India, highlighting market trends.', query='best index funds India awards 2023')]
